# Training Object Detection Model and Extracting Images

Use annotations done by somebody else on a smaller dataset to train the detection model. The extract the ROIs and save in a format that can be used with our setup.

In [ ]:
from IPython.display import clear_output

In [ ]:
%%capture 

!git clone https://github.com/ultralytics/yolov5
    
%cd yolov5
!wandb disabled

In [ ]:
!python train.py --img 1024 --batch 24 --epochs 50 --data /kaggle/input/roi-model-yaml/ROI_model.yml --weights yolov5s.pt --cache --device 0


### Report BBox score on validation

In [ ]:
!python val.py --img 1024 \
--data /kaggle/input/roi-model-yaml/ROI_model.yml \
--weights runs/train/exp/weights/best.pt \
--iou 0.5 --half


### Extract from our data

In [ ]:
import os
import cv2
import torch
from pathlib import Path
from PIL import Image
from torchvision.transforms import ToTensor

# Import YOLOv5
from yolov5.utils.general import non_max_suppression


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt')


In [ ]:
def extract_and_save_roi(image_path, output_path=None):
    img = Image.open(image_path)
    img_tensor = ToTensor()(img).repeat(3, 1, 1)  # Repeat the single channel 3 times
    
    with torch.no_grad():
        results = model(img_tensor.unsqueeze(0), augment=True)
        detections = non_max_suppression(results, conf_thres=0.25, iou_thres=0.45)
    
    if len(detections) > 0:  # Check if there are any detections
        for *xyxy, conf, cls in detections[0]:
            x1, y1, x2, y2 = map(int, xyxy)
            roi = img.crop((x1, y1, x2, y2))
            if output_path != None:
                roi.save(output_path)
            return x1, y1, x2, y2
    else:
        print(f"No detections found for image: {image_path}")
        if output_path != None:
            img.save(output_path)



In [ ]:
input_base_path = '/kaggle/input/rsna-custom-pipeline4/train_images_processed_1024'
output_base_path = '/kaggle/working/train_images_extracted_1024'

Path(output_base_path).mkdir(parents=True, exist_ok=True)


for patient_id in os.listdir(input_base_path):
    patient_input_path = os.path.join(input_base_path, patient_id)
    patient_output_path = os.path.join(output_base_path, patient_id)
    Path(patient_output_path).mkdir(parents=True, exist_ok=True)

    for image_id in os.listdir(patient_input_path):
        input_image_path = os.path.join(patient_input_path, image_id)
        output_image_path = os.path.join(patient_output_path, image_id)
        extract_and_save_roi(input_image_path, output_image_path)


In [ ]:
%matplotlib inline

### Example annotation

In [ ]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np

patient = 17362
image_id = 1712361839




def show_image_with_bbox(image_path, bbox, color='r'):
    img = Image.open(image_path)
    fig, ax = plt.subplots(1)
    ax.imshow(img, cmap='gray')
    
    x1, y1, x2, y2 = bbox
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor=color, facecolor='none')
    ax.add_patch(rect)
    
    plt.axis('off')
    plt.show()

# Replace the path with the actual path to one of the original images
image_path = f'/kaggle/input/rsna-custom-pipeline4/train_images_processed_1024/{patient}/{image_id}.png'

# Replace this with the actual bounding box coordinates
bbox = extract_and_save_roi(image_path)

show_image_with_bbox(image_path, bbox)

### Extracted example

In [ ]:
def show_image(image_path):
    img = Image.open(image_path)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()
image_path = f'/kaggle/working/train_images_extracted_1024/{patient}/{image_id}.png'
show_image(image_path)


### Remove YOLO

In [ ]:
import shutil

# Replace 'your_directory_path' with the path of the directory you want to remove
directory_path = '/kaggle/working/yolov5'

# Remove the directory and all its contents
shutil.rmtree(directory_path)